# Quantum Circuit Builder

In this notebook, I try to make a quantum circuit builder

In [ ]:
# Importing libraries
import numpy as np
import qutip as q
import matplotlib.pyplot as plt

### Numpy

The first approach is to use numpy for everything (that is no QuTip)

In [2]:
# Defining qbit states 0 and 1
bra0 = np.array([[1, 0]])
ket0 = bra0.T
bra1 = np.array([[0, 1]])
ket1 = bra1.T
# Defining identity operator and Pauli operators
I = np.eye(2)
sigx = np.array([[0, 1], [1, 0]])
sigy = np.array([[0, -1j], [1j, 0]])
sigz = np.array([[1, 0], [0, -1]])
# Defining gate operators H, S, T, CNOT, and SWAP
H = np.array([[1, 1], [1, -1]]) / np.sqrt(2)
S = np.array([[1, 0], [0, 1j]])
T = np.array([[1, 0], [0, np.exp(1j * np.pi / 4)]])
CNOT = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])
SWAP = np.array([[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]])

After defining all the relevant gates, I define a function that applies single qubit gates on each qubit and then takes the kronecker product in the end

In [3]:
def QuantumCircuit(states, gates, targets):
    
    """
    Applies a sequence of gates to specified qubits in a quantum state.
    
    Parameters:
    states (numpy.ndarray): The initial quantum states.
    gates (list): A list of gate operators to apply.
    targets (list): A list of target qubit indices for each gate.
    
    Returns:
    numpy.ndarray: The final quantum state after applying all gates.
    """

    # Determine the number of qubits
    N = int(np.size(states)/2)

    # Check if the number of gates matches the number of targets
    if len(gates) != len(targets):
        raise ValueError("The number of gates must match the number of targets.")
    # Check if any of the targets are out of range
    for target in targets:
        if target < 0 or target >= N:
            raise ValueError(f"Target {target} is out of range for {N} qubits.")
    
    for i in range(len(gates)):
        temp = gates[i] @ states[targets[i]]
        states[targets[i]] = temp / np.linalg.norm(temp)  # Normalize the state

    return states

In [4]:
# Define states, gates, and targets
states = [ket0, ket0]  # Initial state |00>
gates = [H] # Apply Hadamard gates to both qubits
targets = [0]  # Apply to both qubits

In [268]:
states

[array([[1],
        [0]]),
 array([[1],
        [0]])]

In [269]:
QuantumCircuit(states, gates, targets)

[array([[0.70710678],
        [0.70710678]]),
 array([[1],
        [0]])]

Okay, this is all fine and well, but we need 2-qubit gates as well. To implement these, one has to use kroneker products for all the unitaries and states.

Now the function bellow is used to generate the unitary from a single-qubit gate.

In [24]:
def generateUnitary(gate, target, N):
    """
    Applies given single-qubit gate to specified qubit positions in an N-qubit system.
    Parameters:
        gate (np.ndarray): Single-qubit gate matrix.
        target (int): Qubit index (1-based). 0 means apply to all qubits.
        N (int): Total number of qubits.
    Returns:
        np.ndarray: Full operator as a Kronecker product.
    """
    
    # Start with identity on all qubits
    operators = [I for _ in range(N)]
    
    # If target is 0, apply the gate to all qubits
    if target == 0:
        # Apply to all qubits
        for i in range(N):
            operators[i] = gate
    else:
        # Apply to specific (1-indexed) qubit
        if not (1 <= target <= N):
            raise ValueError(f"Target index {target} out of bounds for {N} qubits.")
        operators[target - 1] = gate

    # Compute Kronecker product (qubit 1 = leftmost)
    result = operators[0]
    for op in operators[1:]:
        result = np.kron(result, op)
    return result

In [31]:
generateUnitary(gate=H, target=0, N=3)

array([[ 0.35355339,  0.35355339,  0.35355339,  0.35355339,  0.35355339,
         0.35355339,  0.35355339,  0.35355339],
       [ 0.35355339, -0.35355339,  0.35355339, -0.35355339,  0.35355339,
        -0.35355339,  0.35355339, -0.35355339],
       [ 0.35355339,  0.35355339, -0.35355339, -0.35355339,  0.35355339,
         0.35355339, -0.35355339, -0.35355339],
       [ 0.35355339, -0.35355339, -0.35355339,  0.35355339,  0.35355339,
        -0.35355339, -0.35355339,  0.35355339],
       [ 0.35355339,  0.35355339,  0.35355339,  0.35355339, -0.35355339,
        -0.35355339, -0.35355339, -0.35355339],
       [ 0.35355339, -0.35355339,  0.35355339, -0.35355339, -0.35355339,
         0.35355339, -0.35355339,  0.35355339],
       [ 0.35355339,  0.35355339, -0.35355339, -0.35355339, -0.35355339,
        -0.35355339,  0.35355339,  0.35355339],
       [ 0.35355339, -0.35355339, -0.35355339,  0.35355339, -0.35355339,
         0.35355339,  0.35355339, -0.35355339]])

In [271]:
# Define states, gates, and targets
states = [ket0, ket0] # Initial state |00>
gates = [H]
targets = [0]  
N = len(states)

In [273]:
UT = generateUnitary(gates, targets, N)
print(UT)
state = np.kron(states[0], states[1])
result = UT @ state
result /= np.linalg.norm(result)
print(result)

[[ 0.5  0.5  0.5  0.5]
 [ 0.5 -0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5 -0.5  0.5]]
[[0.5]
 [0.5]
 [0.5]
 [0.5]]


This works well. Now it would be nice to expand onto 2-qubit gates.

### QuTip

In [32]:
from qutip import gates
from qutip.qip.operations import *
#from qutip_qip.circuit import QubitCircuit

In [33]:
# Defining qbit states 0 and 1
ket0 = q.basis(2, 0)
bra0 = ket0.dag()
ket1 = q.basis(2, 1)
bra1 = ket1.dag()
# Defining identity operator and Pauli operators
I = q.identity(2)
sigx = q.sigmax()
sigy = q.sigmay()
sigz = q.sigmaz()
# Defining gate operators
H = gates.hadamard_transform()
T = gates.t_gate()
S = gates.s_gate()
CNOT = gates.cnot()
SWAP = gates.swap()

In [34]:
CNOT

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=Dense, isherm=True
Qobj data =
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]